temperatura (e imagino que todos) datos cada 5 minutos

parametros: 
- gente que entra
- temperatura
- 

In [1]:
import requests
import datetime
import time
import pandas as pd

def obtener_datos_api(base_url, sensor_type_names, from_date, to_date, location_ids):
    """
    Extrae datos de una API en periodos de un mes y los guarda en un CSV consolidado.
    
    Cada variable dentro de "series" se convierte en una columna en el DataFrame.
    Cada fila representa un timestamp único con sus valores de diferentes sensores.
    
    :param base_url: URL base de la API.
    :param sensor_type_names: Lista de nombres de sensores a consultar.
    :param from_date: Fecha de inicio en formato 'DD-MM-YYYY HH:MM'.
    :param to_date: Fecha de fin en formato 'DD-MM-YYYY HH:MM'.
    :param location_ids: Lista de IDs de ubicación.
    :return: DataFrame consolidado con todos los datos.
    """
    
    def convertir_a_milisegundos(fecha_str):
        """Convierte una fecha en formato 'DD-MM-YYYY HH:MM' a milisegundos EPOCH."""
        dt = datetime.datetime.strptime(fecha_str, "%d-%m-%Y %H:%M")
        return int(time.mktime(dt.timetuple()) * 1000)
    
    def convertir_a_fecha(epoch_ms):
        """Convierte milisegundos EPOCH a una fecha en formato 'DD-MM-YYYY HH:MM'"""
        return datetime.datetime.fromtimestamp(epoch_ms / 1000).strftime("%d-%m-%Y %H:%M")
    
    from_epoch = convertir_a_milisegundos(from_date)
    to_epoch = convertir_a_milisegundos(to_date)
    
    # Diccionario para consolidar datos antes de convertirlo en DataFrame
    datos_consolidados = {}

    periodo = 30 * 24 * 60 * 60 * 1000  # 1 mes en milisegundos
    
    while from_epoch < to_epoch:
        next_epoch = min(from_epoch + periodo, to_epoch)
        
        params = {
            "sensorTypeNames": ",".join(sensor_type_names),
            "fromDate": from_epoch,
            "toDate": next_epoch,
            "locationIds": ",".join(map(str, location_ids))
        }
        
        response = requests.get(base_url, params=params, verify=False)
        
        if response.status_code == 200:
            data = response.json()
            # Verificamos que la respuesta tenga la clave 'series'
            if "series" in data:
                # Recorremos cada variable dentro de "series"
                for variable, series_dict in data["series"].items():
                    # Cada serie en esta variable puede tener un identificador único
                    for series_id, puntos in series_dict.items():
                        for timestamp, valor in puntos:
                            fecha = convertir_a_fecha(timestamp)
                            # Si la fecha no está en el diccionario, la inicializamos
                            if fecha not in datos_consolidados:
                                datos_consolidados[fecha] = {}
                            # Guardamos el valor en la columna de la variable correspondiente
                            datos_consolidados[fecha][variable] = valor
            else:
                print("La respuesta no contiene la clave 'series'")
        else:
            print(f"Error {response.status_code}: {response.text}")
        
        from_epoch = next_epoch  # Avanzar al siguiente periodo
    
    if not datos_consolidados:
        print("No se obtuvieron datos.")
        return None

    # Convertimos el diccionario en un DataFrame
    df_final = pd.DataFrame.from_dict(datos_consolidados, orient="index")
    df_final.index.name = "timestamp"
    df_final = df_final.reset_index()
    
    # Ordenamos por fecha
    df_final = df_final.sort_values("timestamp")
    
    # Guardamos a CSV
    csv_filename = "datos_consolidados.csv"
    df_final.to_csv(csv_filename, index=False)
    
    return df_final


In [ ]:

url_api = "https://platon.grc.upv.es/sensingtools-api/api/user-hard-sensors/hup4zYsBoTGoLusGWiIj/data" 
sensores = ["count_in", "count_out"]
fecha_inicio = "10-09-2024 00:01"
fecha_fin = "11-12-2024 00:01"
ubicaciones = ['hup4zYsBoTGoLusGWiIj_entrada-outdoor_38.02497_-0.65828_0']

df_final = obtener_datos_api(url_api, sensores, fecha_inicio, fecha_fin, ubicaciones)


C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'platon.grc.upv.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\hugol\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: Insec

              timestamp  count_in  count_out
7929   01-10-2024 00:01       0.0        0.0
7930   01-10-2024 00:06       0.0        0.0
7931   01-10-2024 00:11       0.0        0.0
7932   01-10-2024 00:16       0.0        0.0
7933   01-10-2024 00:21       0.0        0.0
...                 ...       ...        ...
22194  31-10-2024 23:47       0.0        1.0
22195  31-10-2024 23:51       0.0        1.0
22196  31-10-2024 23:52       0.0        1.0
22197  31-10-2024 23:56       0.0        1.0
22198  31-10-2024 23:57       0.0        1.0

[39696 rows x 3 columns]


In [6]:
df_final["count_in"] = df_final["count_in"].astype('Int64')
df_final["count_out"] = df_final["count_out"].astype('Int64')
df_final["people_inside"] = df_final["count_in"] - df_final["count_out"]
df_final

,timestamp,count_in,count_out,people_inside
7929,01-10-2024 00:01,0,0,0
7930,01-10-2024 00:06,0,0,0
7931,01-10-2024 00:11,0,0,0
7932,01-10-2024 00:16,0,0,0
7933,01-10-2024 00:21,0,0,0
...,...,...,...,...
22194,31-10-2024 23:47,0,1,-1
22195,31-10-2024 23:51,0,1,-1
22196,31-10-2024 23:52,0,1,-1
22197,31-10-2024 23:56,0,1,-1
